In [ ]:
# Load all necessary packages
import numpy as np
from numpy import mean
from numpy import std
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.metrics import auc, RocCurveDisplay
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, cross_validate, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Read in pattern and mortality info
patterns=pd.read_csv('patterns.csv')
X = patterns.iloc[:,1:31]
y = patterns[["d28_survival"]]
y = y.values.ravel()

# Create dataframe to hold variable importance data
impdf = pd.DataFrame(X.columns)

# Set up classifier and cross-validation
rf=RandomForestClassifier(class_weight="balanced")
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10)

# Loop through process 10 times
for i in range(0,10):
    # Split into train/test and perform cross-validation
    X_train,X_test,y_train,y_test = train_test_split(X,y,stratify=y,test_size=0.3)
    output = cross_validate(rf, X_train, y_train, cv=cv, scoring='roc_auc', return_estimator =True)
    # Store variable importance at each run
    j = 1
    for idx,estimator in enumerate(output['estimator']):
        impdf.insert(j+i*100, str(j+i*100), estimator.feature_importances_)
        j = j+1
importances = impdf.mean(axis=1)
importancesstd = impdf.std(axis=1)

# Sort by importance, then plot
indices = np.argsort(importances)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='cornflowerblue', align='center',xerr=importancesstd)
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance',fontsize=10)
plt.yticks(fontsize=6.5)
plt.show()

In [ ]:
# Subset dataset to each set of patterns
Xclean = Xnew[['Pattern_23']]
X2clean = Xnew[['Pattern_23','Pattern_4']]
X3clean = Xnew[['Pattern_23','Pattern_4','Pattern_27']]
X4clean = Xnew[['Pattern_23','Pattern_4','Pattern_27','Pattern_15']]
X5clean = Xnew[['Pattern_23','Pattern_4','Pattern_27','Pattern_15','Pattern_12']]
yclean = patterns[["d28_survival"]]
yclean = yclean.values.ravel()

# Set up classifier and cross-validation
classifier = RandomForestClassifier().fit(Xclean,yclean)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10)

# Split data into 70/30 train/test, then perform cross-validation with 10 splits and 10 repeats. Repeat process 10 times
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

fig, ax = plt.subplots()
# Plot line showing "chance" prediction
ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="black", label="Chance", alpha=0.8)
for j in range(0,10):
    X_train,X_test,y_train,y_test = train_test_split(Xclean,yclean,stratify=yclean,test_size=0.3) 
    X = X_test
    y = y_test
    for i, (train, test) in enumerate(cv.split(X, y)):
        classifier.fit(X.iloc[train], y[train])
        viz = RocCurveDisplay.from_estimator(
             classifier,
             X.iloc[test],
             y[test],
             label='_nolegend_',
             alpha=0,
             lw=1,
             ax=ax,
        )
        interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
        interp_tpr[0] = 0.0
        tprs.append(interp_tpr)
        aucs.append(viz.roc_auc)
    # Average curves from cross-validation and multiple train/test splits
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
# Plot average curve
ax.plot(
    mean_fpr,
    mean_tpr,
    color="cornflowerblue",
    label=r"P23 (AUC = %0.3f $\pm$ %0.3f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

# Repeat same process for each set of patterns
classifier = RandomForestClassifier().fit(X2clean,yclean)

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

for j in range(0,10):
    X_train,X_test,y_train,y_test = train_test_split(X2clean,yclean,stratify=yclean,test_size=0.3) 
    X2 = X_test
    y = y_test
    for i, (train, test) in enumerate(cv.split(X2, y)):
     classifier.fit(X2.iloc[train], y[train])
     viz = RocCurveDisplay.from_estimator(
         classifier,
         X2.iloc[test],
         y[test],
         label='_nolegend_',
         alpha=0,
         lw=1,
         ax=ax,
     )
     interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
     interp_tpr[0] = 0.0
     tprs.append(interp_tpr)
     aucs.append(viz.roc_auc)
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="gold",
    label=r"P23+P4 (AUC = %0.3f $\pm$ %0.3f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

classifier = RandomForestClassifier().fit(X3clean,yclean)

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

for j in range(0,10):
    X_train,X_test,y_train,y_test = train_test_split(X3clean,yclean,stratify=yclean,test_size=0.3) 
    X3 = X_test
    y = y_test
    for i, (train, test) in enumerate(cv.split(X3, y)):
     classifier.fit(X3.iloc[train], y[train])
     viz = RocCurveDisplay.from_estimator(
         classifier,
         X3.iloc[test],
         y[test],
         label='_nolegend_',
         alpha=0,
         lw=1,
         ax=ax,
     )
     interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
     interp_tpr[0] = 0.0
     tprs.append(interp_tpr)
     aucs.append(viz.roc_auc)
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="orangered",
    label=r"P23+P4+P27 (AUC = %0.3f $\pm$ %0.3f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

classifier = RandomForestClassifier().fit(X4clean,yclean)

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

for j in range(0,10):
    X_train,X_test,y_train,y_test = train_test_split(X4clean,yclean,stratify=yclean,test_size=0.3) 
    X4 = X_test
    y = y_test
    for i, (train, test) in enumerate(cv.split(X4, y)):
     classifier.fit(X4.iloc[train], y[train])
     viz = RocCurveDisplay.from_estimator(
         classifier,
         X4.iloc[test],
         y[test],
         label='_nolegend_',
         alpha=0,
         lw=1,
         ax=ax,
     )
     interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
     interp_tpr[0] = 0.0
     tprs.append(interp_tpr)
     aucs.append(viz.roc_auc)


    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="springgreen",
    label=r"P23+P4+P27+P15 (AUC = %0.3f $\pm$ %0.3f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

classifier = LogisticRegression(max_iter=1000).fit(X5clean,yclean)

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

for j in range(0,10):
    X_train,X_test,y_train,y_test = train_test_split(X5clean,yclean,stratify=yclean,test_size=0.3) 
    X5 = X_test
    y = y_test
    for i, (train, test) in enumerate(cv.split(X5, y)):
     classifier.fit(X5.iloc[train], y[train])
     viz = RocCurveDisplay.from_estimator(
         classifier,
         X5.iloc[test],
         y[test],
         label='_nolegend_',
         alpha=0,
         lw=1,
         ax=ax,
     )
     interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
     interp_tpr[0] = 0.0
     tprs.append(interp_tpr)
     aucs.append(viz.roc_auc)


    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="blueviolet",
    label=r"P23+P4+P27+P15+P12 (AUC = %0.3f $\pm$ %0.3f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

ax.set(
xlim=[-0.05, 1.05],
ylim=[-0.05, 1.05],
title="ROC Curves Across Pattern Sets",
xlabel="False Positive Rate",
ylabel="True Positive Rate"
)

ax.set_aspect('equal')
ax.legend(loc='lower right',prop={'size': 6})
plt.show()

In [1]:
# Subset data to relevant patterns
X = patterns[["Pattern_23","Pattern_4","Pattern_27","Pattern_15","Pattern_12"]]
y = patterns[["d28_survival"]]
y = y.values.ravel()

# Set up logistic regression model
res = sm.Logit(y,X).fit()
params = res.params
# Get 95% CIs of coefficients
conf = res.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['5%', '95%', 'Odds Ratio']
# Exponentiate coefficients to get odds ratios
print(np.exp(conf))

NameError: name 'patterns' is not defined